<a href="https://colab.research.google.com/github/suryanshgupta-creator/projects/blob/main/Earnings_Call_Complexity_Index_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Earnings Call Complexity Index
### A Data Science Project

## 1. Introduction
- Motivation: Do companies use more complex language when delivering bad news?
- Hypothesis: Firms in trouble use longer, vaguer, more complex language.
- Dataset: Earnings call transcripts (S&P 500).
- Goal: Build a Complexity Index and test its relationship with stock performance.

In [ ]:
# Install necessary libraries
!pip install textstat lexicalrichness vaderSentiment yfinance datasets

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk, spacy, re
import textstat
from lexicalrichness import LexicalRichness
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf

# Load spacy model (run once if not already)
# !python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

## 2. Load and Explore Data

In [ ]:
# Example loading transcripts CSV
from datasets import load_dataset

ds = load_dataset("lamini/earnings-calls-qa")
df = ds["train"].to_pandas()
df.head()

## 3. Preprocessing Functions

In [ ]:
# Sample 10 unique companies
unique_companies = df["ticker"].unique()
if len(unique_companies) > 100:
    sampled_companies = np.random.choice(unique_companies, 10, replace=False)
    df_sampled = df[df["ticker"].isin(sampled_companies)].copy()
else:
    df_sampled = df.copy()

display(df_sampled.head())

In [ ]:
# Describe the sampled dataset
print("Shape of the sampled dataframe:", df_sampled.shape)
print("\nInfo of the sampled dataframe:")
df_sampled.info()
print("\nDescription of the sampled dataframe:")
display(df_sampled.describe(include='all'))

In [ ]:
def preprocess_text(text):
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    words = [token.text.lower() for token in doc if token.is_alpha]
    return sentences, words, doc


## 4. Complexity Metrics

In [ ]:
def avg_sentence_length(sentences):
    return np.mean([len(s.split()) for s in sentences])

def readability_scores(text):
    return textstat.flesch_kincaid_grade(text)

def lexical_diversity(words):
    lex = LexicalRichness(" ".join(words))
    return lex.mtld()

def jargon_ratio(words, jargon_list):
    return sum(1 for w in words if w in jargon_list) / len(words)

def hedging_ratio(words, hedge_list):
    return sum(1 for w in words if w in hedge_list) / len(words)

def passive_voice_ratio(doc):
    return sum(1 for tok in doc if tok.dep_ in ("auxpass","nsubjpass")) / len(list(doc.sents))


## 5. Apply Metrics to Dataset

In [ ]:
# Load spacy model
import spacy
import textstat
nlp = spacy.load("en_core_web_sm")

# Assuming LexicalRichness is already imported from lexicalrichness (from a previous successful cell)
from lexicalrichness import LexicalRichness
import numpy as np # import numpy

def lexical_diversity(words):
    lex = LexicalRichness(" ".join(words))
    return lex.mtld()

def avg_sentence_length(sentences): # define avg_sentence_length
    return np.mean([len(s.split()) for s in sentences])

def readability_scores(text): # define readability_scores
    return textstat.flesch_kincaid_grade(text)

def jargon_ratio(words, jargon_list): # define jargon_ratio
    return sum(1 for w in words if w in jargon_list) / len(words)

def hedging_ratio(words, hedge_list): # define hedging_ratio
    return sum(1 for w in words if w in hedge_list) / len(words)

def passive_voice_ratio(doc): # define passive_voice_ratio
    return sum(1 for tok in doc if tok.dep_ in ("auxpass","nsubjpass")) / len(list(doc.sents))

# define preprocess_text
def preprocess_text(text):
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    words = [token.text.lower() for token in doc if token.is_alpha]
    return sentences, words, doc


jargon_list = ["synergy","leverage","efficiency","ecosystem","strategic","roadmap"]
hedge_list = ["might","could","may","anticipated","believe","expected"]

results = []
for _, row in df_sampled.iterrows():
    sentences, words, doc = preprocess_text(row["transcript"])
    metrics = {
        "company": row["ticker"],
        "date": row["date"],
        "sentence_len": avg_sentence_length(sentences),
        "readability": readability_scores(row["transcript"]),
        "lexical_div": lexical_diversity(words),
        "jargon_ratio": jargon_ratio(words, jargon_list),
        "hedge_ratio": hedging_ratio(words, hedge_list),
        "passive_ratio": passive_voice_ratio(doc),
        "ticker": row["ticker"] # Add the ticker column
    }
    results.append(metrics)

df_metrics = pd.DataFrame(results)
display(df_metrics.head())

In [ ]:
!pip install lexicalrichness

## 6. Normalize Metrics & Compute Complexity Index

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
cols = ["sentence_len","readability","lexical_div","jargon_ratio","hedge_ratio","passive_ratio"]
df_metrics[cols] = scaler.fit_transform(df_metrics[cols])

def compute_index(row):
    return (
        0.25*row["readability"] +
        0.20*row["sentence_len"] +
        0.15*(1-row["lexical_div"]) +
        0.15*row["jargon_ratio"] +
        0.15*row["hedge_ratio"] +
        0.10*row["passive_ratio"]
    )

df_metrics["complexity_index"] = df_metrics.apply(compute_index, axis=1)
df_metrics.sort_values("complexity_index", ascending=False).head(10)

## 7. Link to Stock Performance

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from pandas.tseries.offsets import BDay

def compute_forward_returns(df_metrics, ticker_col="ticker", date_col="date", days=5):
    """
    Vectorized forward-returns calculator:
      1) Downloads all prices in one call.
      2) Aligns to next business day.
      3) Computes (Close[t+days] / Close[t]) - 1.
    Returns a copy of df_metrics with a new column: f"return_{days}d".
    """
    # --- 0) Basic hygiene ---
    df = df_metrics.copy()
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce', format='mixed')
    # normalize tickers (important if you pass ticker_col="company")
    df[ticker_col] = df[ticker_col].astype(str).str.strip().str.upper()

    ret_col = f"return_{days}d"

    # --- 1) Global date window & unique tickers ---
    tickers = sorted(df[ticker_col].dropna().unique().tolist())
    if not tickers:
        df[ret_col] = np.nan
        return df

    dmin = df[date_col].min()
    dmax = df[date_col].max()
    # buffer for weekends/holidays & forward window
    start_dl = (dmin - BDay(2)).date()
    end_dl   = (dmax + BDay(days + 5)).date()

    # --- 2) Download once for all tickers ---
    px = yf.download(
        tickers=tickers,
        start=start_dl,
        end=end_dl,
        auto_adjust=True,
        progress=False,
        group_by="ticker",
        threads=True
    )

    # If nothing came back, return NaNs (avoids hangs later)
    if px is None or len(px) == 0:
        df[ret_col] = np.nan
        return df

    # --- 3) Build {ticker: Close series} map ---
    close_map = {}
    if isinstance(px.columns, pd.MultiIndex):
        # multi-ticker shape
        for t in tickers:
            key = (t, "Close")
            if key in px.columns:
                s = px[key].dropna().copy()
                s.name = t
                close_map[t] = s
    else:
        # single-ticker shape
        if "Close" in px.columns:
            t = tickers[0]
            s = px["Close"].dropna().copy()
            s.name = t
            close_map[t] = s

    # --- 4) Helper: compute forward return from preloaded prices ---
    def row_forward_return(tick, dt):
        s = close_map.get(tick)
        if s is None or s.empty or pd.isna(dt):
            return np.nan

        # next business day as start; t+days business days as end
        start_dt = (pd.to_datetime(dt) + BDay(1)).normalize()
        end_dt   = start_dt + BDay(days)

        # Use integer positions produced by searchsorted (works on DatetimeIndex)
        idx0 = s.index.searchsorted(start_dt)
        idx1 = s.index.searchsorted(end_dt)

        if idx0 >= len(s) or idx1 >= len(s):
            return np.nan

        p0 = s.iloc[idx0]
        p1 = s.iloc[idx1]
        if pd.isna(p0) or pd.isna(p1):
            return np.nan

        return (p1 - p0) / p0

    # --- 5) Compute returns without any network calls in the loop ---
    df[ret_col] = [row_forward_return(t, d) for t, d in zip(df[ticker_col], df[date_col])]
    return df

# -------- USE IT --------
# If your "company" column holds ticker symbols, this is correct:
df_metrics = compute_forward_returns(df_metrics, ticker_col="company", date_col="date", days=5)
# df_metrics now has:  df_metrics["return_5d"]
display(df_metrics.head())

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr

# --- Add "complexity_index" to the metrics list ---
complexity_vars = [
    "sentence_len", "readability", "lexical_div",
    "jargon_ratio", "hedge_ratio", "passive_ratio",
    "complexity_index"
]

# Drop rows missing any of these columns or return_5d
df_corr = df_metrics.dropna(subset=["return_5d"] + complexity_vars)

# --- Create 4x2 grid to accommodate all 7 plots neatly ---
rows = 4
cols = 2
fig, axes = plt.subplots(rows, cols, figsize=(12, 16))
axes = axes.flatten()

for i, var in enumerate(complexity_vars):
    ax = axes[i]
    x = df_corr[var]
    y = df_corr["return_5d"]

    # Compute correlation
    r, p = pearsonr(x, y)

    # Scatter plot
    ax.scatter(x, y, alpha=0.5, color="steelblue", edgecolor="none")

    # Regression line
    if len(x.unique()) > 1:  # ensure variance
        coef = np.polyfit(x, y, 1)
        xs = np.linspace(x.min(), x.max(), 200)
        ys = np.poly1d(coef)(xs)
        ax.plot(xs, ys, color="black", linewidth=1.5)

    # Add zero-return reference line
    ax.axhline(0, color="gray", linestyle="--", linewidth=1)

    # Highlight statistically significant correlations
    title_color = "red" if p < 0.05 else "black"
    ax.set_title(f"{var} vs Return\n(r={r:.2f}, p={p:.3f})", color=title_color, fontsize=11)

    ax.set_xlabel(var)
    ax.set_ylabel("5-Day Return")

# Hide any unused subplot slots if < rows*cols
for j in range(len(complexity_vars), len(axes)):
    fig.delaxes(axes[j])

plt.suptitle("Correlation between Complexity Factors (including Composite Index) and 5-Day Stock Returns",
             fontsize=14, y=1.02)
plt.tight_layout()
plt.show()




In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

# Ensure date is datetime and convert to numeric (for regression)
df_metrics["date"] = pd.to_datetime(df_metrics["date"])
df_plot = df_metrics.dropna(subset=["complexity_index"]).copy()
df_plot["date_ordinal"] = df_plot["date"].map(pd.Timestamp.toordinal)

top_companies = (
    df_plot["company"].value_counts().head(10).index
)
plt.figure(figsize=(10,6))
for company, group in df_plot[df_plot["company"].isin(top_companies)].groupby("company"):
    plt.plot(group["date"], group["complexity_index"], marker="o", label=company)
plt.title("Top 5 Companies – Complexity Index Over Time")
plt.xlabel("Date")
plt.ylabel("Complexity Index")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
top_companies = (
    df_plot["company"].value_counts().head(10).index
)
plt.figure(figsize=(10,6))
for company, group in df_plot[df_plot["company"].isin(top_companies)].groupby("company"):
    plt.plot(group["date"], group["complexity_index"], marker="o", label=company)
plt.title("Top 5 Companies – Complexity Index Over Time")
plt.xlabel("Date")
plt.ylabel("Complexity Index")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

# Ensure date is datetime and convert to numeric (for regression)
df_metrics["date"] = pd.to_datetime(df_metrics["date"])
df_plot = df_metrics.dropna(subset=["complexity_index"]).copy()
df_plot["date_ordinal"] = df_plot["date"].map(pd.Timestamp.toordinal)

# Compute slope for each company
trend_data = []
for company, group in df_plot.groupby("company"):
    if len(group) < 3:  # skip if too few points
        continue
    X = group["date_ordinal"].values.reshape(-1, 1)
    y = group["complexity_index"].values
    model = LinearRegression().fit(X, y)
    slope = model.coef_[0]
    trend_data.append({"company": company, "slope": slope})

df_trends = pd.DataFrame(trend_data)

# Add interpretation columns
df_trends["trend_direction"] = np.where(df_trends["slope"] > 0, "Increasing", "Decreasing")
df_trends["abs_slope"] = df_trends["slope"].abs()

# Sort by strength of trend
df_trends = df_trends.sort_values("slope", ascending=False)
display(df_trends.head(10))

# Plot complexity index over time for top companies
top_companies = (
    df_plot["company"].value_counts().head(10).index
)
plt.figure(figsize=(10,6))
for company, group in df_plot[df_plot["company"].isin(top_companies)].groupby("company"):
    plt.plot(group["date"], group["complexity_index"], marker="o", label=company)
plt.title("Top 5 Companies – Complexity Index Over Time")
plt.xlabel("Date")
plt.ylabel("Complexity Index")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
top_increasing = df_trends.nlargest(5, "slope")
top_decreasing = df_trends.nsmallest(5, "slope")

plt.figure(figsize=(10,5))
plt.barh(top_increasing["company"], top_increasing["slope"], color="red", label="More Complex")
plt.barh(top_decreasing["company"], top_decreasing["slope"], color="green", label="More Clear")
plt.axvline(0, color="black", linewidth=1)
plt.xlabel("Complexity Trend (Slope)")
plt.title("Companies with Strongest Changes in Complexity Over Time")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Compute average 5-day stock return per company
avg_returns = (
    df_metrics.groupby("company")["return_5d"]
    .mean()
    .reset_index()
    .rename(columns={"return_5d": "avg_return_5d"})
)

# Merge trend data with average return
df_combined = pd.merge(df_trends, avg_returns, on="company", how="inner")

display(df_combined.head())


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8,6))
sns.regplot(
    x="slope",
    y="avg_return_5d",
    data=df_combined,
    scatter_kws={"alpha":0.7},
    line_kws={"color":"black"}
)

plt.axhline(0, color="gray", linestyle="--", linewidth=1)
plt.axvline(0, color="gray", linestyle="--", linewidth=1)
plt.title("Do Clarity Trends Correlate with Stock Performance?")
plt.xlabel("Complexity Trend Slope (↑ = More Complex, ↓ = More Clear)")
plt.ylabel("Average 5-Day Stock Return")
plt.tight_layout()
plt.show()


In [ ]:
from scipy.stats import pearsonr

r, p = pearsonr(df_combined["slope"], df_combined["avg_return_5d"])
print(f"Correlation between complexity trend and avg 5-day return: r = {r:.3f}, p = {p:.4f}")


## 8. Visualizations

In [ ]:
# Company ranking
df_sorted = df_metrics.sort_values("complexity_index")
plt.figure(figsize=(8,6))
plt.barh(df_sorted["company"], df_sorted["complexity_index"])
plt.xlabel("Complexity Index")
plt.title("Company Ranking by Complexity Index")
plt.show()


## 9. Time-series overlay

In [ ]:
# Example: Time-series overlay for one company
company = "Apple"
apple_df = df_metrics[df_metrics["company"]==company]
plt.plot(apple_df["date"], apple_df["complexity_index"], marker="o")
plt.title(f"{company}: Complexity Index Over Time")
plt.xlabel("Quarter")
plt.ylabel("Complexity Index")
plt.show()


## 10. Results & Insights
- Complex language often linked to weaker stock returns.
- Sector-level differences visible.
- Hedging and jargon are strong predictors.

## 11. Conclusion
- Language complexity reflects more than style — it may signal financial stress.
- Clear communication often aligns with stronger performance.
- Future: Expand dataset, apply advanced NLP models.